In [ ]:
#using Markdown; Markdown.parse_file("README.md")

In [ ]:
MODEL_PATH_KEY = :wNB;
const FLTYPE = Float32;
include("heavyloadings.jl");

In [ ]:
half_b = load(MODEL_FOLDER*"\\bunny\\half_bunny_unmod.ply");
half_b_rot = load(MODEL_FOLDER*"\\bunny\\half_bunny_rot1.ply");

In [ ]:
include("firsticp.jl")

In [ ]:
half_b_sc = scaleMesh(half_b,10);
half_b_rot_sc = scaleMesh(half_b_rot,10);

In [ ]:
vis = Visualizer();

In [ ]:
open(vis)
#IJuliaCell(vis)

In [ ]:
placeMeshInLife(vis,half_b_sc,"hbunny","darkred");

In [ ]:
placeMeshInLife(vis,half_b_rot_sc,"rotbunny","olivedrab2")

## New workflow
Using arrays (bármilyen lehessen: SArray, CuArray, Array)

Ötlet: amíg lehet csak az az indexeket használni és csak ahol muszáj, ott számolni a konkrét mátrixokkal
### Workflow lépései:
1. [x] reading és ref mesh-t tömbbé alakítani
1. [x] a teljes referencia tömbből kd-tree
### Iteráció
1. [x] reading tömbből sampling
1. [x] samplinged tömbhöz párkeresés a fából
1. ##### Worst `x`% rejection
  * [x] párok sorbarendezése távolság szerint
  * [x] a sorbarendezett párok `x`%-nak kidobása
1. ##### 2.5sigma rejection
  * [ ] szórás számítása
  * [ ] 2.5 sigmánál nagyobbak kidobása
1. [x] kvaternió magic
1. [x] transzformáció a megoldással
1. [ ] transzformáció mentése vizualizációhoz
1. [ ] kilépési feltétel check

### Iteráció vége

11. [ ] place in MeshCat
12. [ ] animate in MeshCat


## TODOs:
* vizualizáció
* animáció

### Newnew workflow with matrix instead of array of array

In [ ]:
#Workflow: (reading, reference)
#Input meshes/pointclouds
reference_pc = deepcopy(half_b_sc);
reading_pc = noisifyMesh(half_b_rot_sc);

#Creating matrixes
ref_mat = makeMatrixFromMesh(vertices(reference_pc));
red_mat = makeMatrixFromMesh(vertices(reading_pc));

#Transposed reference matrix
ref_mtp = giveTranspose(ref_mat)
red_mtp = giveTranspose(red_mat)

#Building KD tree for the whole ref array (transposed matrix must be used)
#kdtree_ref = KDTree(ref_mtp);

In [ ]:
#si_red = randomSampleIndexes(50, red_mtp, 2);
#pairing the points in the two meshes
#indM, traitM = createKnnPairArray(red_mtp[:,si_red], kdtree_ref);
#sorting the indexes by the distances in the trait matrix
#dist_arr = @view traitM[:,2]
#s_indM, s_it = sortIndexes(indM, dist_arr);
#remove the worst %
#ch_ind, ch_it = chopEndOfArray(20, s_indM,s_it);

In [ ]:
#Make some iterations
#input: ref_mat, red_mat
function itera(reff,redd,itnum)
    reffTP = giveTranspose(reff)
    reddTP = giveTranspose(redd)
    kdtree_ref = KDTree(reffTP)
    #iterate from here
    for i in 1:itnum
        si = randomSampleIndexes(20,size(reffTP,1))
        indM, traitM = createKnnPairArray(reffTP[:,si], kdtree_ref)
        dist_arr = @view traitM[:,2]
        s_indM, s_it = sortIndexes(indM, dist_arr)
        #remove the worst %
        ch_ind, ch_it = chopEndOfArray(20, s_indM,s_it)
        ref_view = @view reff[s_indM[1:end,1],:]
        red_view = @view redd[s_indM[1:end,2],:]
        #@show ref_view
        global R = bestRot(red_view,ref_view)
        mu_ref = CoM(reff)
        mu_red = CoM(redd)
        global qt = mu_ref - R * mu_red
        
    end
    return R, qt
end

# Again new

In [ ]:
#input: reading és reference
#homogén koordináták: oszlopvektorok egymás mellé helyezve
reference_pc = deepcopy(half_b_sc);
reading_pc = noisifyMesh(half_b_rot_sc);
#Homogeneous vectors
refA = convert2HomCoordMatrix(vertices(reference_pc));
redA = convert2HomCoordMatrix(vertices(reading_pc));
#kd tree
kdtree_ref = KDTree(refA);

In [ ]:
#here starts the iteration
pairPC = 30; #using the pairPC percent of the pairs
rmPC = 20; #remove the rmPC % worst pairs
ri = randomSampleIndexes(pairPC,refA,2); #random indexes
indM, traitM = createKnnPairArray(redA[:,ri], kdtree_ref); #pairing the points, here @view can't be used
#different rejection methods come here
    distA = @view traitM[:,2];
    sindM, sit = sortIndexes(indM, distA);
    #reject the % worst
    chind, chit = chopEndOfArray(rmPC, sindM, sit);
    refV = @view refA[1:3,chind[1:end,1]];
    redV = @view redA[1:3,chind[1:end,2]];
#refV and redV contains the paired and filtered points
mu_ref = CoM(refV, 1);
mu_red = CoM(redV, 1);
# cross-cov. mat.
sigma = crosscovMat(redV, refV, mu_red, mu_ref);
# the best rotation matrix
rMat = bestRotMat(sigma);
# best translation vector
bTr = mu_ref - rMat*mu_red;
homTR  = [rMat bTr ; 0 0 0 1]

In [ ]:
#p is the reading, x is the reference
#the inverse of the returned matrix should be used
function useAll(p, x)
    mu_p = CoM(p,1)
    mu_x = CoM(x,1)
    Sigma = crosscovMat(p, x, mu_p, mu_x)
    rM = bestRotMat(Sigma)
    bTr = mu_x - rM*mu_p
    return [rM bTr; 0 0 0 1]
end   

In [ ]:
# dummy test
hMat = useAll(refA[1:3,:],redA[1:3,:])
r2Arr =  inv(hMat) * redA
rtMi = makeMeshfromMatrix(r2Arr[1:3,:])
placeMeshInLife(vis, rtMi, "inversed", "orange")